In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


In [3]:
MAX_SEQUENCE_LENGTH = 5000
MAX_NUM_WORDS = 25000
EMBEDDING_DIM = 300
TEST_SPLIT = 0.2
TEXT_DATA = 'dataset.csv'

In [4]:
df = pd.read_csv(TEXT_DATA)
df.drop(labels=['id','title'], axis='columns', inplace=True)
# only select stories with lengths gt 0 -- there are some texts with len = 0
mask = list(df['text'].apply(lambda x: len(x) > 0))
df = df[mask]
print(df)

                                                   text label
0     Daniel Greenfield, a Shillman Journalism Fello...  FAKE
1     Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE
2     U.S. Secretary of State John F. Kerry said Mon...  REAL
3     — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE
4     It's primary day in New York and front-runners...  REAL
...                                                 ...   ...
6330  The State Department told the Republican Natio...  REAL
6331  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...  FAKE
6332   Anti-Trump Protesters Are Tools of the Oligar...  FAKE
6333  ADDIS ABABA, Ethiopia —President Obama convene...  REAL
6334  Jeb Bush Is Suddenly Attacking Trump. Here's W...  REAL

[6335 rows x 2 columns]


In [5]:
texts = df['text']
labels = df['label']

print('Found %s texts.' %texts.shape[0])

Found 6335 texts.


In [77]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
print(sequences)

word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
data = pad_sequences(sequences, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='pre', 
                     truncating='pre')

print('Found %s unique tokens.' % len(word_index))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Found 98817 unique tokens.
Shape of data tensor: (6335, 5000)
Shape of label tensor: (6335,)


In [7]:

from sklearn.model_selection import train_test_split

train_text_data, test_text_data, train_labels, test_labels = train_test_split(data, 
                                                  labels.apply(lambda x: 0 if x == 'FAKE' else 1),
                                                  test_size=TEST_SPLIT)


In [8]:
from keras import layers
from keras.models import Sequential

model = Sequential(
    [
        # part 1: word and sequence processing
        layers.Embedding(num_words,
                         EMBEDDING_DIM, 
                         input_length=MAX_SEQUENCE_LENGTH,
                         trainable=True),
        layers.Conv1D(128, 5, activation='relu'),
        layers.GlobalMaxPooling1D(),
        
        # part 2: classification
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5000, 300)         7500300   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4996, 128)         192128    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 7,709,069
Trainable params: 7,709,069
Non-trainable params: 0
_________________________________________________________________


In [9]:
history = model.fit(train_text_data, 
                    train_labels,
                    batch_size=128,
                    epochs=6,
                    validation_data=(test_text_data, test_labels))


Train on 5068 samples, validate on 1267 samples
Epoch 1/6
5068/5068 [==============================] - 124s 24ms/step - loss: 0.4661 - acc: 0.7820 - val_loss: 0.2522 - val_acc: 0.8998
Epoch 2/6
5068/5068 [==============================] - 123s 24ms/step - loss: 0.1295 - acc: 0.9584 - val_loss: 0.1215 - val_acc: 0.9487
Epoch 3/6
5068/5068 [==============================] - 125s 25ms/step - loss: 0.0385 - acc: 0.9917 - val_loss: 0.0935 - val_acc: 0.9613
Epoch 4/6
5068/5068 [==============================] - 126s 25ms/step - loss: 0.0078 - acc: 0.9996 - val_loss: 0.0904 - val_acc: 0.9597
Epoch 5/6
5068/5068 [==============================] - 125s 25ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0934 - val_acc: 0.9597
Epoch 6/6
5068/5068 [==============================] - 129s 25ms/step - loss: 1.8269e-04 - acc: 1.0000 - val_loss: 0.0979 - val_acc: 0.9637


In [17]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [18]:
def predict(X):
    return np.rint(model.predict(X))

In [19]:
for i in range(100):
    q = predict( np.array( [train_text_data[i],] )  )
    print(q)




[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]


In [21]:
real = """U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sunday’s unity march against terrorism.

Kerry said he expects to arrive in Paris Thursday evening, as he heads home after a week abroad. He said he will fly to France at the conclusion of a series of meetings scheduled for Thursday in Sofia, Bulgaria. He plans to meet the next day with Foreign Minister Laurent Fabius and President Francois Hollande, then return to Washington.

The visit by Kerry, who has family and childhood ties to the country and speaks fluent French, could address some of the criticism that the United States snubbed France in its darkest hour in many years.

The French press on Monday was filled with questions about why neither President Obama nor Kerry attended Sunday’s march, as about 40 leaders of other nations did. Obama was said to have stayed away because his own security needs can be taxing on a country, and Kerry had prior commitments.

Among roughly 40 leaders who did attend was Israeli Prime Minister Benjamin Netanyahu, no stranger to intense security, who marched beside Hollande through the city streets. The highest ranking U.S. officials attending the march were Jane Hartley, the ambassador to France, and Victoria Nuland, the assistant secretary of state for European affairs. Attorney General Eric H. Holder Jr. was in Paris for meetings with law enforcement officials but did not participate in the march.

Kerry spent Sunday at a business summit hosted by India’s prime minister, Narendra Modi. The United States is eager for India to relax stringent laws that function as barriers to foreign investment and hopes Modi’s government will act to open the huge Indian market for more American businesses.

In a news conference, Kerry brushed aside criticism that the United States had not sent a more senior official to Paris as “quibbling a little bit.” He noted that many staffers of the American Embassy in Paris attended the march, including the ambassador. He said he had wanted to be present at the march himself but could not because of his prior commitments in India.

“But that is why I am going there on the way home, to make it crystal clear how passionately we feel about the events that have taken place there,” he said.

“And I don’t think the people of France have any doubts about America’s understanding of what happened, of our personal sense of loss and our deep commitment to the people of France in this moment of trauma.”"""

In [22]:
real

'U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sunday’s unity march against terrorism.\n\nKerry said he expects to arrive in Paris Thursday evening, as he heads home after a week abroad. He said he will fly to France at the conclusion of a series of meetings scheduled for Thursday in Sofia, Bulgaria. He plans to meet the next day with Foreign Minister Laurent Fabius and President Francois Hollande, then return to Washington.\n\nThe visit by Kerry, who has family and childhood ties to the country and speaks fluent French, could address some of the criticism that the United States snubbed France in its darkest hour in many years.\n\nThe French press on Monday was filled with questions about why neither President Obama nor Kerry attended Sunday’s march, as about 40 leaders of other nations did. Obama was said to have stayed away because his own security needs can be taxing on a country, 

In [23]:
fake = """Google Pinterest Digg Linkedin Reddit Stumbleupon Print Delicious Pocket Tumblr 
There are two fundamental truths in this world: Paul Ryan desperately wants to be president. And Paul Ryan will never be president. Today proved it. 
In a particularly staggering example of political cowardice, Paul Ryan re-re-re-reversed course and announced that he was back on the Trump Train after all. This was an aboutface from where he was a few weeks ago. He had previously declared he would not be supporting or defending Trump after a tape was made public in which Trump bragged about assaulting women. Suddenly, Ryan was appearing at a pro-Trump rally and boldly declaring that he already sent in his vote to make him President of the United States. It was a surreal moment. The figurehead of the Republican Party dosed himself in gasoline, got up on a stage on a chilly afternoon in Wisconsin, and lit a match. . @SpeakerRyan says he voted for @realDonaldTrump : “Republicans, it is time to come home” https://t.co/VyTT49YvoE pic.twitter.com/wCvSCg4a5I 
— ABC News Politics (@ABCPolitics) November 5, 2016 
The Democratic Party couldn’t have asked for a better moment of film. Ryan’s chances of ever becoming president went down to zero in an instant. In the wreckage Trump is to leave behind in his wake, those who cravenly backed his campaign will not recover. If Ryan’s career manages to limp all the way to 2020, then the DNC will have this tape locked and loaded to be used in every ad until Election Day. 
The ringing endorsement of the man he clearly hates on a personal level speaks volumes about his own spinelessness. Ryan has postured himself as a “principled” conservative, and one uncomfortable with Trump’s unapologetic bigotry and sexism. However, when push came to shove, Paul Ryan – like many of his colleagues – turned into a sniveling appeaser. After all his lofty tak about conviction, his principles were a house of cards and collapsed with the slightest breeze. 
What’s especially bizarre is how close Ryan came to making it through unscathed. For months the Speaker of the House refused to comment on Trump at all. His strategy seemed to be to keep his head down, pretend Trump didn’t exist, and hope that nobody remembered what happened in 2016. Now, just days away from the election, he screwed it all up. 
If 2016’s very ugly election has done any good it’s by exposing the utter cowardice of the Republicans who once feigned moral courage. A reality television star spit on them, hijacked their party, insulted their wives, and got every last one of them to kneel before him. What a turn of events. 
Featured image via Twitter"""

In [24]:
fake

'Google Pinterest Digg Linkedin Reddit Stumbleupon Print Delicious Pocket Tumblr \nThere are two fundamental truths in this world: Paul Ryan desperately wants to be president. And Paul Ryan will never be president. Today proved it. \nIn a particularly staggering example of political cowardice, Paul Ryan re-re-re-reversed course and announced that he was back on the Trump Train after all. This was an aboutface from where he was a few weeks ago. He had previously declared he would not be supporting or defending Trump after a tape was made public in which Trump bragged about assaulting women. Suddenly, Ryan was appearing at a pro-Trump rally and boldly declaring that he already sent in his vote to make him President of the United States. It was a surreal moment. The figurehead of the Republican Party dosed himself in gasoline, got up on a stage on a chilly afternoon in Wisconsin, and lit a match. . @SpeakerRyan says he voted for @realDonaldTrump : “Republicans, it is time to come home” ht

In [47]:
sequences = tokenizer.texts_to_sequences(real)

prediction = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, 500) + 1
inputdata = pad_sequences(sequences, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='pre', 
                     truncating='pre')


In [51]:
predict( np.array( [inputdata[1502],] )  )

array([[0.]], dtype=float32)

In [59]:
from keras.preprocessing.text import Tokenizer
tok = Tokenizer()
# train tokenizer
tok.fit_on_texts([real]) 
# print sequences
ughdata = tok.texts_to_sequences([real])[0]
print(tok.texts_to_sequences([real])[0]) 
print(tok.texts_to_sequences([real])[0])

[35, 36, 37, 3, 38, 75, 76, 7, 10, 39, 6, 5, 18, 77, 4, 12, 78, 40, 41, 79, 19, 6, 42, 80, 20, 21, 22, 43, 81, 11, 82, 83, 7, 10, 5, 84, 2, 85, 4, 12, 44, 86, 15, 5, 87, 45, 88, 8, 41, 89, 5, 10, 5, 18, 90, 2, 13, 23, 1, 91, 3, 8, 92, 3, 46, 93, 14, 44, 4, 94, 95, 5, 96, 2, 97, 1, 98, 99, 24, 47, 25, 100, 101, 9, 48, 102, 49, 103, 104, 2, 105, 1, 106, 50, 7, 26, 107, 108, 9, 109, 110, 2, 1, 51, 9, 111, 112, 52, 53, 113, 114, 3, 1, 19, 6, 1, 27, 28, 115, 13, 4, 116, 117, 118, 4, 54, 119, 1, 52, 120, 29, 39, 16, 121, 24, 122, 17, 55, 123, 48, 56, 124, 7, 22, 43, 11, 15, 17, 57, 58, 3, 125, 126, 30, 56, 16, 10, 2, 31, 127, 128, 59, 60, 129, 61, 130, 131, 62, 132, 29, 8, 51, 9, 7, 32, 63, 64, 133, 134, 57, 58, 26, 30, 135, 16, 136, 65, 25, 137, 138, 42, 139, 2, 140, 61, 26, 141, 142, 49, 143, 1, 144, 145, 1, 146, 147, 35, 36, 21, 148, 1, 11, 149, 150, 151, 1, 66, 2, 13, 9, 152, 153, 1, 154, 37, 3, 38, 14, 155, 156, 157, 158, 159, 160, 161, 162, 16, 4, 12, 14, 46, 24, 163, 164, 21, 67, 30, 

In [60]:
ughdata

[35,
 36,
 37,
 3,
 38,
 75,
 76,
 7,
 10,
 39,
 6,
 5,
 18,
 77,
 4,
 12,
 78,
 40,
 41,
 79,
 19,
 6,
 42,
 80,
 20,
 21,
 22,
 43,
 81,
 11,
 82,
 83,
 7,
 10,
 5,
 84,
 2,
 85,
 4,
 12,
 44,
 86,
 15,
 5,
 87,
 45,
 88,
 8,
 41,
 89,
 5,
 10,
 5,
 18,
 90,
 2,
 13,
 23,
 1,
 91,
 3,
 8,
 92,
 3,
 46,
 93,
 14,
 44,
 4,
 94,
 95,
 5,
 96,
 2,
 97,
 1,
 98,
 99,
 24,
 47,
 25,
 100,
 101,
 9,
 48,
 102,
 49,
 103,
 104,
 2,
 105,
 1,
 106,
 50,
 7,
 26,
 107,
 108,
 9,
 109,
 110,
 2,
 1,
 51,
 9,
 111,
 112,
 52,
 53,
 113,
 114,
 3,
 1,
 19,
 6,
 1,
 27,
 28,
 115,
 13,
 4,
 116,
 117,
 118,
 4,
 54,
 119,
 1,
 52,
 120,
 29,
 39,
 16,
 121,
 24,
 122,
 17,
 55,
 123,
 48,
 56,
 124,
 7,
 22,
 43,
 11,
 15,
 17,
 57,
 58,
 3,
 125,
 126,
 30,
 56,
 16,
 10,
 2,
 31,
 127,
 128,
 59,
 60,
 129,
 61,
 130,
 131,
 62,
 132,
 29,
 8,
 51,
 9,
 7,
 32,
 63,
 64,
 133,
 134,
 57,
 58,
 26,
 30,
 135,
 16,
 136,
 65,
 25,
 137,
 138,
 42,
 139,
 2,
 140,
 61,
 26,
 141,
 142,
 49,
 143,
 

In [72]:
for i in range (500):
    p = predict( np.array( [data[i],] )  )
    print(p)

[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[1.]]
[[0.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[1.]]
[[0.]]
[[1.]]
[[1.]]
[[0.]]

In [66]:
len(inputdata[44])

5000

In [67]:
print(data[0])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [79]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
#sequences = tokenizer.texts_to_sequences(real)
print(sequences)
#word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
data = pad_sequences(sequences, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='pre', 
                     truncating='pre')
print((data))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

